In [57]:
import numpy as np
import xarray as xr
import pandas as pd
import os, glob

import matplotlib.pyplot as plt
import cartopy.crs as ccrs

In [2]:
import dask
from dask_jobqueue import PBSCluster
from dask.distributed import Client
ncores=1
nmem='20GB'
cluster = PBSCluster(
    cores=ncores, # The number of cores you want
    memory=nmem, # Amount of memory
    processes=1, # How many processes
    queue='casper', # The type of queue to utilize (/glade/u/apps/dav/opt/usr/bin/execcasper)
    local_directory='$TMPDIR', # Use your local directory
    resource_spec='select=1:ncpus='+str(ncores)+':mem='+nmem, # Specify resources
    account='P93300041', # Input your project ID here
    walltime='03:00:00', # Amount of wall time
    #interface='ib0', # Interface to use
)

# Scale up
cluster.scale(10)

# Setup your client
client = Client(cluster)

/glade/work/linnia/conda-envs/mlenv/lib/python3.10/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42539 instead
  warnings.warn(
Task exception was never retrieved
future: <Task finished name='Task-441153' coro=<Client._gather.<locals>.wait() done, defined at /glade/work/linnia/conda-envs/mlenv/lib/python3.10/site-packages/distributed/client.py:2394> exception=AllExit()>
Traceback (most recent call last):
  File "/glade/work/linnia/conda-envs/mlenv/lib/python3.10/site-packages/distributed/client.py", line 2403, in wait
    raise AllExit()
distributed.client.AllExit


In [108]:
def amean(da,cf=1/365):
    #annual mean
    m  = da['time.daysinmonth']
    xa = cf*(m*da).groupby('time.year').sum().compute()
    xa.name=da.name
    return xa

In [3]:
# csv file with ensemble info
df_crosswalk = pd.read_csv('/glade/u/home/czarakas/coupled_PPE/code/02_set_up_ensemble/CLM5PPE_coupledPPE_crosswalk.csv')

### Atmospheric variables

In [116]:
# load ensemble:  atmospheric variables
in_dir = '/glade/campaign/cgd/tss/people/czarakas/CoupledPPE/coupled_simulations/'
midfix = '_PI_SOM_v02/atm/proc/tseries/month_1/'
suffix = '_PI_SOM_v02.cam.h0.'

# List of variables to loop over
dvs = ['PRECC', 'PRECL', 'PRECSC', 'PRECSL', 'LHFLX', 'SHFLX', 'SWCF', 'TS']

# Filter dataframe
df_filtered = df_crosswalk[df_crosswalk['param'] != 'default']
params = df_filtered['param'].unique()

# List to collect datasets for each param
ds_list = []

for param in params:
    df_param = df_filtered[df_filtered['param'] == param]
    
    # Get key_coupledPPE for min and max
    key_min = df_param[df_param['minmax'] == 'min']['key_coupledPPE'].values[0]
    key_max = df_param[df_param['minmax'] == 'max']['key_coupledPPE'].values[0]
    
    param_dict = {}
    
    for var in dvs:
        # Build file patterns (with wildcard *)
        file_min_pattern = in_dir + key_min + midfix + key_min + suffix + var + '*.nc'
        file_max_pattern = in_dir + key_max + midfix + key_max + suffix + var + '*.nc'
        
        # Open datasets
        ds_min = xr.open_mfdataset(file_min_pattern, combine='by_coords')
        ds_max = xr.open_mfdataset(file_max_pattern, combine='by_coords')
        
        # Extract variable and average over time
        da_min = amean(ds_min[var]).mean(dim='year')
        da_max = amean(ds_max[var]).mean(dim='year')

        # Stack min and max manually along new 'minmax' dimension
        da = xr.concat([da_min, da_max], dim=pd.Index(['min', 'max'], name='minmax'))
        
        # Expand along param dimension
        da = da.expand_dims(param=[param])
        
        param_dict[var] = da
    
    # Create a dataset for this param
    ds_param = xr.Dataset(param_dict)
    
    # Append to list
    ds_list.append(ds_param)

# Now combine all parameters into one dataset
ds_atm = xr.concat(ds_list, dim='param')

ds_atm['RAIN'] = (ds_atm['PRECC']+ds_atm['PRECL'])
ds_atm['SNOW'] = (ds_atm['PRECSC']+ds_atm['PRECSL'])
ds_atm['TOTPREC'] = (ds_atm['RAIN']+ds_atm['SNOW'])


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x1538e1ed0220>>
Traceback (most recent call last):
  File "/glade/work/linnia/conda-envs/mlenv/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
# load default atm variables
in_dir = '/glade/campaign/cgd/tss/people/czarakas/CoupledPPE/coupled_simulations/COUP0000_PI_SOM/atm/proc/tseries/month_1/'

# Loop over variables and files
dvs = ['PRECC', 'PRECL', 'PRECSC', 'PRECSL', 'LHFLX', 'SHFLX', 'SWCF', 'TS']

default_atm = xr.Dataset()
for var in dvs:
        
    file = glob.glob(in_dir + 'COUP0000_PI_SOM.cam.h0.' +var +'.'+ '*.nc')[0]
    ds = xr.open_dataset(file)
    default_atm[var] = amean(ds[var]).mean(dim='year')

default_atm['RAIN'] = (default_atm['PRECC']+default_atm['PRECL'])
default_atm['SNOW'] = (default_atm['PRECSC']+default_atm['PRECSL'])
default_atm['TOTPREC'] = (default_atm['RAIN']+default_atm['SNOW'])


In [ ]:
atm_configs=[
    {'var':'RAIN', 'cf':60*60*24, 'units':'(mm/day)', 'range': 0.5},
    {'var':'SNOW', 'cf':60*60*24, 'units':'(mm/day)', 'range': 0.5},
    {'var':'TOTPREC', 'cf':60*60*24, 'units':'(mm/day)', 'range': 0.5},
    {'var':'LHFLX', 'cf':1, 'units':'(W/m2)', 'range': 10},
    {'var':'SHFLX', 'cf':1, 'units':'(W/m2)', 'range': 10},
    {'var':'SWCF', 'cf':1, 'units':'(W/m2)', 'range': 3},
    {'var':'TS', 'cf':1, 'units':'(K)', 'range': 2},
]

In [ ]:
### Snow
# Constants
cf = 60 * 60 * 24 * 1000  # mm/day

# Calculate snow as (PRECSC + PRECSL)
data = (ds_final['PRECSC'] + ds_final['PRECSL'])
default = default_atm['SNOW']

# Set up the polar stereographic projection
proj = ccrs.NorthPolarStereo()

fig, axes = plt.subplots(
    nrows=6, ncols=6, 
    subplot_kw={'projection': proj}, 
    figsize=(24, 16)
)

# Flatten axes array for easy iteration
axes = axes.flatten()

# Loop over parameters and minmax
param_values = ds_final['param'].values
minmax_values = ['min', 'max']

plot_idx = 0
for param in param_values:
    for mm in minmax_values:
        ax = axes[plot_idx]
        
        # Select rain for this param and min/max
        var_param = data.sel(param=param, minmax=mm)
    
        # Compute the anomaly
        var = var_param - default
        
        # Plot
        pc = ax.pcolormesh(
            ds_final['lon'], ds_final['lat'], cf * var,
            vmin=-0.5, vmax=0.5,
            cmap='BrBG',
            transform=ccrs.PlateCarree(),
            shading='auto'
        )
        
        ax.set_title(str(param)+' '+mm+' - default')
        ax.set_extent([-180, 180, 50, 90], crs=ccrs.PlateCarree())
        ax.coastlines()
        plot_idx += 1
    
    # Add a colorbar
    fig.subplots_adjust(bottom=0.1, top=0.9)
    cbar_ax = fig.add_axes([0.25, 0.05, 0.5, 0.02])  # [left, bottom, width, height]
    cbar = fig.colorbar(pc, cax=cbar_ax, orientation='horizontal')
    cbar.set_label('SNOW (mm/day)')

plt.tight_layout()
plt.savefig('./figs/coupledPPE_snow.png')


### Land model variables

In [112]:
# load ensemble land variables
in_dir = '/glade/campaign/cgd/tss/people/czarakas/CoupledPPE/coupled_simulations/'
midfix = '_PI_SOM_v02/lnd/proc/tseries/month_1/'
suffix = '_PI_SOM_v02.clm2.h0.'

# List of variables to loop over
dvs = ['QRUNOFF','VPD2M','TLAI','GPP','ALTMAX','QDRAI','QDRAI_PERCH','QSNOMELT','QH2OSFC','TSOI_10CM','SNOW','RAIN']

# Filter dataframe
df_filtered = df_crosswalk[df_crosswalk['param'] != 'default']
params = df_filtered['param'].unique()

# List to collect datasets for each param
ds_list = []

for param in params:
    df_param = df_filtered[df_filtered['param'] == param]
    
    # Get key_coupledPPE for min and max
    key_min = df_param[df_param['minmax'] == 'min']['key_coupledPPE'].values[0]
    key_max = df_param[df_param['minmax'] == 'max']['key_coupledPPE'].values[0]
    
    param_dict = {}
    
    for var in dvs:
        # Build file patterns (with wildcard *)
        file_min_pattern = glob.glob(in_dir + key_min + midfix + key_min + suffix + var +'.'+ '*.nc')[0]
        file_max_pattern = glob.glob(in_dir + key_max + midfix + key_max + suffix + var +'.'+ '*.nc')[0]
        
        # Open datasets
        ds_min = xr.open_dataset(file_min_pattern)
        ds_max = xr.open_dataset(file_max_pattern)
        
        # Extract variable and average over time
        da_min = amean(ds_min[var]).mean(dim='year')
        da_max = amean(ds_max[var]).mean(dim='year')

        # Stack min and max manually along new 'minmax' dimension
        da = xr.concat([da_min, da_max], dim=pd.Index(['min', 'max'], name='minmax'))
        
        # Expand along param dimension
        da = da.expand_dims(param=[param])
        
        param_dict[var] = da
    
    # Create a dataset for this param
    ds_param = xr.Dataset(param_dict)
    
    # Append to list
    ds_list.append(ds_param)

# Now combine all parameters into one dataset
ds_land = xr.concat(ds_list, dim='param')


In [113]:
# load default land variables
in_dir = '/glade/campaign/cgd/tss/people/czarakas/CoupledPPE/coupled_simulations/COUP0000_PI_SOM/lnd/proc/tseries/month_1/'

# Loop over variables and files
default_land = xr.Dataset()
for var in dvs:
        
    file = glob.glob(in_dir + 'COUP0000_PI_SOM.clm2.h0.' +var +'.'+ '*.nc')[0]
    ds = xr.open_dataset(file)
    default_land[var] = amean(ds[var]).mean(dim='year')


In [117]:
configs=[
    {'var':'QRUNOFF', 'cf':60*60*24*365, 'units':'(mm/year)', 'range': 100},
    {'var':'VPD2M', 'cf':1/1000, 'units':'(kPa)', 'range': 5},
    {'var':'TLAI', 'cf':1, 'units':'(m2/m2)', 'range': 1.5},
    {'var':'GPP', 'cf':60*60*24, 'units':'(gC/m2/day)', 'range': 3},
    {'var':'ALTMAX', 'cf':1, 'units':'(m)', 'range': 1},
    {'var':'QDRAI', 'cf':60*60*24*365, 'units':'(mm/year)', 'range': 100},
    {'var':'QDRAI_PERCH', 'cf':60*60*24*365, 'units':'(mm/year)', 'range': 100},
    {'var':'QSNOMELT', 'cf':60*60*24*365, 'units':'(mm/year)', 'range': 100},
    {'var':'QH2OSFC', 'cf':60*60*24*365, 'units':'(mm/year)', 'range': 100},
    {'var':'TSOI_10CM', 'cf':1, 'units':'(K)', 'range': 5},
    {'var':'SNOW', 'cf':60*60*24, 'units':'(mm/day)', 'range': 0.5},
    {'var':'RAIN', 'cf':60*60*24, 'units':'(mm/day)', 'range': 0.5},
]

In [ ]:
### Plot PPE-default for all land variables

for i in range(len(configs)):
    var = configs[i]['var']
    data = ds_land[var]
    default = default_land[var]
    
    # Set up the polar stereographic projection
    proj = ccrs.NorthPolarStereo()
    
    fig, axes = plt.subplots(
        nrows=6, ncols=6, 
        subplot_kw={'projection': proj}, 
        figsize=(24, 16)
    )
    
    # Flatten axes array for easy iteration
    axes = axes.flatten()
    
    # Loop over parameters and minmax
    param_values = ds_final['param'].values
    minmax_values = ['min', 'max']
    
    plot_idx = 0
    for param in param_values:
        for mm in minmax_values:
            ax = axes[plot_idx]
            
            # Select rain for this param and min/max
            var_param = data.sel(param=param, minmax=mm)
        
            # Compute the anomaly
            delta = var_param - default
            
            # Plot
            pc = ax.pcolormesh(
                ds_land['lon'], ds_land['lat'], configs[i]['cf'] * delta,
                vmin=-configs[i]['range'], vmax=configs[i]['range'],
                cmap='BrBG',
                transform=ccrs.PlateCarree(),
                shading='auto'
            )
            
            ax.set_title(str(param)+' '+mm+' - default')
            ax.set_extent([-180, 180, 50, 90], crs=ccrs.PlateCarree())
            ax.coastlines()
            plot_idx += 1
        
        # Add a colorbar
        fig.subplots_adjust(bottom=0.1, top=0.9)
        cbar_ax = fig.add_axes([0.25, 0.05, 0.5, 0.02])  # [left, bottom, width, height]
        cbar = fig.colorbar(pc, cax=cbar_ax, orientation='horizontal')
        cbar.set_label(var + ' ' + configs[i]['units'])
    
    
    plt.savefig('./figs/coupledPPE_'+var+'.png')
